In [1]:
knitr::opts_chunk$set(echo = TRUE)
library(ggplot2)
library(caret)
library(ranger)
library(lattice)
library(rpart)
library(rattle)
library(glmnet)

Loading required package: lattice
Rattle: A free graphical interface for data science with R.
Version 5.2.0 Copyright (c) 2006-2018 Togaware Pty Ltd.
Type 'rattle()' to shake, rattle, and roll your data.

Attaching package: 'rattle'

The following object is masked from 'package:ranger':

    importance

Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-16



In [2]:
df <- read.csv("bank-full.csv")

names(df) <- c("age", "jobClass", "maritalStatus", "educationLevel", "creditDefault", "defaultBalance", "housingLoan", "personalLoan", "communicationType", "lastDayContacted", "lastMonthContact", "lastContactDuration", "totalContacts", "daysLastContact", "previousContacts", "previousOutcome", "target")

Warning message in file(file, "rt"):
"cannot open file 'bank-full.csv': No such file or directory"

ERROR: Error in file(file, "rt"): cannot open the connection


In [3]:
tr <- ggplot(df, aes(df$target)) + geom_bar() + ggtitle("Target Variable Distribution")
a <- ggplot(df, aes(df$age, color = df$target)) + geom_density() + ggtitle("Density Plot: Age by Campaign Success")
b <- ggplot(df, aes(df$target)) + geom_bar(aes(fill = df$jobClass)) + ggtitle("Campaign Success by Job")
c <- ggplot(df, aes(df$target)) + geom_bar(aes(fill = df$maritalStatus)) + ggtitle("Campaign Success by Marital Status")
d <- ggplot(df, aes(df$target)) + geom_bar(aes(fill = df$educationLevel)) + ggtitle("Campaign Success by Education Level")
e <- ggplot(df, aes(df$target)) + geom_bar(aes(fill = df$creditDefault)) + ggtitle("Campaign Success by Credit Default")
f <- ggplot(df, aes(df$defaultBalance, color = df$target)) + geom_density() + ggtitle("Density Plot: Default Balance by Campaign Success")
g <- ggplot(df, aes(df$target)) + geom_bar(aes(fill = df$housingLoan)) + ggtitle("Campaign Success by Housing Loan Status")
h <- ggplot(df, aes(df$target)) + geom_bar(aes(fill = df$personalLoan)) + ggtitle("Campaign Success by Personal Loan Status")
i <- ggplot(df, aes(df$target)) + geom_bar(aes(fill = df$communicationType)) + ggtitle("Campaign Success by Preferred Communication Method")
j <- ggplot(df, aes(df$lastContactDuration, color = df$target)) + geom_density() + ggtitle("Density Plot: Campaign Success by Last Contact Duration")
k <- ggplot(df, aes(df$target)) + geom_bar(aes(fill = df$previousOutcome)) + ggtitle("Campaign Success by Previous Campaign Success")
l <- ggplot(df, aes(df$totalContacts, color = df$target)) + geom_density() + ggtitle("Density Ploy: Total Contacts by Previous Campaign Success")
m <- ggplot(df, aes(df$previousContacts, color = df$target)) + geom_density() + ggtitle("Density Plot: Previous Contacts by Campaign Success")
n <- ggplot(df, aes(x=df$defaultBalance, y=df$lastContactDuration, color = df$target)) + geom_point() + ggtitle("Default Balance by Last Contact Duration")
o <- ggplot(df, aes(x=df$age, y=df$lastContactDuration, color = df$target)) + geom_point() + ggtitle("Age by LastContact Duration")
p <- ggplot(df, aes(x=df$creditDefault, y=df$defaultBalance)) + stat_summary(mapping = aes(fill = df$target), fun.y ="mean", geom = "bar") + ggtitle("Average Credit Balance By Default Status")
q <- ggplot(df, aes(x=df$target, y=df$lastContactDuration)) + stat_summary(fun.y ="mean", geom = "bar") + ggtitle("Average Contact Duration by Campaign Success")
r <- ggplot(df, aes(df$housingLoan)) + geom_bar(aes(fill = df$personalLoan)) + ggtitle("Housing Loan by Personal Loan")

ERROR: Error: You're passing a function as global data.
Have you misspelled the `data` argument in `ggplot()`


In [4]:
df <- as.data.frame(lapply(df, function(cc) cc[ sample(c(TRUE, NA), prob = c(0.95, 0.05), size = length(cc), replace = TRUE) ]))

'Method 1'
repnum = function(x){
  for(i in 1:ncol(x)){
    if(is.numeric(x[,i])){
     x[is.na(x[,i]), i] <- mean(x[,i], na.rm = TRUE)
    }
  }
  return(x)
}
df = repnum(df)

'Method 2'
repcat = function(x){
  for(i in 1:ncol(x)){
    if(is.factor(x[,i])){
     x[,i][is.na(x[,i])] = levels(x[,i])[which.max(table(x[,i]))]
    }
  }
  return(x)
}
df = repcat(df)

'Method 3'
df <- df[complete.cases(df), ]

ERROR: Error in cc[sample(c(TRUE, NA), prob = c(0.95, 0.05), size = length(cc), : object of type 'symbol' is not subsettable


In [5]:
levels(df$jobClass) <- c("whiteCollar", "blueCollar", "whiteCollar", "blueCollar", "whiteCollar", "Other", "Mixed", "Mixed", "Other", "blueCollar", "Other", "Other")

ERROR: Error in `*tmp*`$jobClass: object of type 'closure' is not subsettable


In [6]:
levels(df$lastMonthContact) <- c("Spring", "Summer", "Winter", "Winter", "Winter", "Summer", "Summer", "Spring", "Spring", "Fall", "Fall", "Fall")

ERROR: Error in `*tmp*`$lastMonthContact: object of type 'closure' is not subsettable


In [7]:
levels(df$creditDefault) <- c("1", "0")
df$creditDefault <- as.numeric(df$creditDefault)

ERROR: Error in `*tmp*`$creditDefault: object of type 'closure' is not subsettable


In [8]:
levels(df$educationLevel) <- c("1", "2", "3", "1")
df$educationLevel <- as.numeric(df$educationLevel)

ERROR: Error in `*tmp*`$educationLevel: object of type 'closure' is not subsettable


In [9]:
splitIndex <- createDataPartition(df$target, p = .70, list = FALSE, times = 1)
train <- df[ splitIndex,]
test <- df[-splitIndex,]

train1 <- train[train$target == "yes",]
train0 <- train[train$target == "no",]

n0 = nrow(train0)
n1 = nrow(train1)
train00 <- train0[sample(1:n0, n1),]
train_under = rbind(train00, train1)

ERROR: Error in df$target: object of type 'closure' is not subsettable


In [10]:
splitIndex <- createDataPartition(df$target, p = .70, list = FALSE, times = 1)
train <- df[ splitIndex,]
test <- df[-splitIndex,]

train1 <- train[train$target == "yes",]
train0 <- train[train$target == "no",]

n0 = nrow(train0)
n1 = nrow(train1)
train11 <- train1[sample(1:n1, n0, replace = TRUE),]
train_over = rbind(train11,train0)

ERROR: Error in df$target: object of type 'closure' is not subsettable


In [11]:
set.seed(2018)
model_under = ranger(target~., data = train_under)
pred_under = predict(model_under, data = test)$predictions
cm_under = confusionMatrix(pred_under, test$target, positive="yes")
cm_under
prop.table(table(train_under$target))

ERROR: Error in "gwaa.data" %in% class(data): object 'train_under' not found


In [12]:
set.seed(2018)
mytree <- rpart(target ~ ., data = train_under, method = "class")
pred_under <- predict(mytree, test, type = "class")
fancyRpartPlot(mytree)
cm = confusionMatrix(data = pred_under, reference = test$target, positive = "yes")
cm

ERROR: Error in is.data.frame(data): object 'train_under' not found


In [13]:
set.seed(2018)
glmdef = train(target~., train_under, method = "glmnet")
max(glmdef$results$Accuracy)

ERROR: Error in eval(expr, p): object 'train_under' not found


In [14]:
set.seed(2018)
myGridglm = expand.grid(alpha = 0:1, lambda = seq(0.0001, 0.1, length = 10))
myControl = trainControl(method = "cv", number = 7)
glmtuned = train(target~., train_under, method ="glmnet",tuneGrid = myGridglm, trControl = myControl)
max(glmtuned$results$Accuracy)

ERROR: Error in eval(expr, p): object 'train_under' not found


In [15]:
set.seed(2018)
model_over = ranger(target~., data = train_over)
pred_over = predict(model_over, data = test)$predictions
cm_over = confusionMatrix(pred_over, test$target, positive="yes")
cm_over
prop.table(table(train_over$target))

ERROR: Error in "gwaa.data" %in% class(data): object 'train_over' not found


In [16]:
set.seed(2018)
mytree <- rpart(target ~ ., data = train_over, method = "class")
pred_over <- predict(mytree, test, type = "class")
fancyRpartPlot(mytree)
cm = confusionMatrix(data = pred_over, reference = test$target, positive = "yes")
cm

ERROR: Error in is.data.frame(data): object 'train_over' not found


In [17]:
set.seed(2018)
glmdef = train(target~., train_over, method = "glmnet")
max(glmdef$results$Accuracy)

ERROR: Error in eval(expr, p): object 'train_over' not found


In [18]:
set.seed(2018)
myGridglm = expand.grid(alpha = 0:1, lambda = seq(0.0001, 0.1, length = 10))
myControl = trainControl(method = "cv", number = 7)
glmtuned = train(target~., train_over, method ="glmnet",tuneGrid = myGridglm, trControl = myControl)
max(glmtuned$results$Accuracy)

ERROR: Error in eval(expr, p): object 'train_over' not found


In [19]:
set.seed(2018)
myGrid <- expand.grid(mtry = 7, splitrule = c("gini","extratrees"), min.node.size = c(1:5))
model <- train(target ~.,data = train_under, method = "ranger", 
               trControl = trainControl(method ="cv", number = 7, verboseIter = TRUE),
               tuneGrid = myGrid)
model

ERROR: Error in eval(expr, p): object 'train_under' not found
